# Orders EDA

```{note} 

`orders_dow` variable means "orders day of the week". Each day corresponds to a number as follows:

:::{list-table} 
:widths: 10 90
:header-rows: 1

*   - Number
    - Day of the week
*   - 0
    - Saturday
*   - 1 
    - Sunday
*   - 2
    - Monday 
*   - 3 
    - Tuesday
*   - 4 
    - Wednesday
*   - 5 
    - Thursday
*   - 6 
    - Friday 
:::

```

## 01 Setup

In [1]:
#Import libriaries
import numpy as np
import pandas as pd 
import os

In [2]:
# Path
path = r'/Users/peanutcookie/instacart-book/'

In [3]:
# Import .csv file
df = pd.read_csv(os.path.join(path, '_csv-raw', 'orders.csv'), index_col = False)

## 02 Dataframe exploration 

In [4]:
# Dataframe shape
df.shape

(3421083, 7)

In [5]:
# Dataframe head
df.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
# Dataframe tail
df.tail(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0
3421082,272231,206209,train,14,6,14,30.0


In [7]:
# Return DataFrame columns names and types of the data their store
df.dtypes

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

## 03 Dataframe Cleansing

### Uniforming Axis Entries

**Dropping columns** 

eval_set column is required for machine learning purposes. SET is one of the four following evaluation sets (eval_set in orders):

- `"prior"` orders prior to that users most recent order (~3.2m orders)
- `"train"` training data supplied to participants (~131k orders)
- `"test"` test data reserved for machine learning competitions (~75k orders)

This project doesn't focus on machine learning procedures, and the column can be dropped.

In [8]:
# Drop eval_set col
df = df.drop(columns = ['eval_set'])

**Unifying columns names**

In [9]:
df.rename(columns = {'order_dow' : 'order_day_of_week'}, inplace = True)

In [10]:
df.head(1)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN


### Data types

In [11]:
df.dtypes

order_id                    int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

In [12]:
## Correcting dataypes - assignig Str data type to order_id, user_id, order_number to receive correct staistics records
df = df.astype({"order_id":'str', "user_id":'str', "order_number":'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   order_number            int64  
 3   order_day_of_week       int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 156.6+ MB


### Missing values

In [13]:
# Return missing values
df.isna().sum()

order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

There is **206209 NaN values** in `days_since_prior_order` column. The value means initial oder after a user's signup. NaN here has a meaning which indicates "new customer", so we cannot remove missing values. </br>

However, we can create a new column `new_customer` of bool data type, which will be `True` for a new customer if `NaN` and `False` otherwise.

In [14]:
# Adress missing values in the days_since_prior_order column
df['new_customer'] = df['days_since_prior_order'].isnull() == True
df

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,False
3421079,1854736,206209,11,4,10,30.0,False
3421080,626363,206209,12,1,12,18.0,False
3421081,2977660,206209,13,1,12,7.0,False


### Duplicates

In [15]:
# Return duplicates
duplicated_rows = df[df.duplicated()]
duplicated_rows.shape

(0, 7)

### Mixed values

In [16]:
# Search and return mixed Values
print("Mixed data")
for col in df.columns.tolist():
  mixed = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[mixed]) > 0:
    print (col + "True")
  else :
    print(col + ": False")

Mixed data
order_id: False
user_id: False
order_number: False
order_day_of_week: False
order_hour_of_day: False
days_since_prior_order: False
new_customer: False


In [17]:
df.head(5)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False


In [18]:
df.shape

(3421083, 7)

In [19]:
df.dtypes

order_id                   object
user_id                    object
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
new_customer                 bool
dtype: object

## 04 Export 

In [20]:
df.to_pickle(os.path.join(path, '_database', 'orders.pkl'))
